In [38]:
import io
import traceback
import opensim as osim
from connections import AWS
import xml.etree.ElementTree as ET
from biomech.processing.trc import *

In [91]:
""" INITIALIZE AWS CONNECTION """
aws_connection = AWS()
aws_connection.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


$\textbf{Inverse Kinematics: Pipeline}$

- Conservative window: BR - 0.217 seconds; then add 0.25 seconds on either side for later trimming
- Runs tool with updated template

In [40]:
from biomech.algorithms import diff_three_point

In [41]:
## OUTLINE

# connect to AWS
# load all subject info
# iterate through subjects:
    # load scaled model, IK template --> write to local dir
    # load trial TRC file --> write to local dir
    # setup & run IK tool 


# OTHER NEEDS

# ball release identification function
# trial .mot file to upload
# trial errors

In [42]:
# load a .mot file (e.g., results from an IK run)
def load_mot_file(path: str) -> pd.DataFrame:
    return pd.read_csv(path, delim_whitespace=True, skiprows=10)

In [47]:
# load all subject info
subject_info = aws_connection.load_subject_info()

# load all FILTERED mot files
    # NOTE: this is a recent change to ensure clean joint angles make it to ID
    # it's not entirely clear how to the OpenSim filtering process works, so this is a safety measure
s3_objects = aws_connection.list_s3_objects(prefix='biomechanics/subjects/')
mot_files = [obj for obj in s3_objects if obj.endswith('.mot') and '_ik_filtered' in obj]

# set XML dir path 
XML_DIR = 'biomechanics/xml_templates'

In [ ]:
# initialize error log, ball release times
error_log = []

# iterate through subjects
for subject in subject_info['subject_id'].unique():
    
    try:
        # get subject info (ID -- string; throwing hand)
        subject_id = str(subject)
        throwing_hand = subject_info[subject_info['subject_id'] == subject]['throws'].values[0]

        # list all subject trials (in trc_processed) & iterate through
        subject_trials = [t for t in mot_files if f'{subject_id}' in t]
        
        if len(subject_trials) > 0:
            print(f'Processing subject {subject_id} with {len(subject_trials)} trials...')

            # load scaled model, save to local dir
            scaled_model = aws_connection.load_xml_from_s3(f'biomechanics/subjects/{subject_id}/osim/{subject_id}_scaled_model.osim')
            default_model_tree = ET.ElementTree(scaled_model)
            default_model_tree.write(f'trial_model.osim')
            
            # update id template with scaled model, output motion file
            id_tree_raw = ET.ElementTree('trial_template.xml')
            id_root = id_tree_raw.getroot()

            # iterate through trials
            for trial in subject_trials:
                # load trial IK results file (.mot) to local dir
                aws_connection.s3.download_file(
                    aws_connection.bucket_name, 
                    trial, 
                    'trial_motion.mot'
                )
                trial_mot_data = load_mot_file('trial_motion.mot')

                """ ID SETUP """
                # set id window (subtract 0.217 seconds, then ± 0.25 seconds on either side)
                id_start_time = trial_mot_data['time'].values[0]
                id_end_time = trial_mot_data['time'].values[-1]

                """ id RUNNING """
                # initialize id tool
                id_tool = osim.InverseDynamicsTool(f'trial_template.xml')
                
                # update start/end time
                id_tool.setStartTime(id_start_time)
                id_tool.setEndTime(id_end_time)

                # set ID cutoff frequency
                id_tool.setLowpassCutoffFrequency(18)  # this is not BW; see https://pmc.ncbi.nlm.nih.gov/articles/PMC4340741/
                
                # run tool
                id_tool.run()

                """ UPLOADS TO S3 """
                # set trial ID from file
                trial_id = trial.split('/')[-1].split('_ik')[0]

                # upload to subject folder (non CSV)
                aws_connection.s3.upload_file(
                    'trial_moments.sto', 
                    aws_connection.bucket_name,
                    f'biomechanics/subjects/{subject_id}/inverse_dynamics/{trial_id}_id.sto'
                )
                aws_connection.s3.upload_file(
                    f'trial_template.xml', 
                    aws_connection.bucket_name,
                    f'biomechanics/subjects/{subject_id}/inverse_dynamics/{trial_id}_id_template.xml'
                )

                # upload results to S3 (CSV)
                trial_moments_df = pd.read_csv('trial_moments.sto', sep='\s+', skiprows=6)
                aws_connection.upload_to_s3(
                    trial_moments_df, 
                    f'biomechanics/subjects/{subject_id}/inverse_dynamics/{trial_id}_id_results.csv'
                )

            print(f'Completed processing for subject {subject_id}.')

    except Exception as e:
        print(f"Error processing subject {subject_id}: {e}")

        traceback.print_exc()

        # log error
        error_log.append({
            'subject_id': subject_id,
            'error': str(e)
        })

        continue

In [93]:
# concatenate error log
error_log_df = pd.DataFrame(error_log)

# write error log to S3 bucket
aws_connection.upload_to_s3(
    error_log_df.to_csv(index=False),
    'biomechanics/subjects/summary/error_id_log.csv'
)

[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/summary/error_id_log.csv


In [94]:
# close AWS connection
aws_connection.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
